<a href="https://colab.research.google.com/github/riya-chougule/Image_segmentation_for_self_driving_cars_using_deep_learning_techniques/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
from fastai.vision import *
import wandb
from wandb.fastai import WandbCallback
from functools import partialmethod

In [2]:
wandb.init(project="semantic-segmentation")

# Define hyper-parameters
config = wandb.config           # for shortening
config.framework = "fast.ai"    # AI framework used (for when we create other versions)
config.img_size = (360, 640)    # dimensions of resized image - can be 1 dim or tuple
config.batch_size = 4           # Batch size during training
config.epochs = 20              # Number of epochs for training
encoder = models.resnet18       
config.encoder = encoder.__name__
config.pretrained = True        # whether we use a frozen pre-trained encoder
config.weight_decay = 1e-4      # weight decay applied on layers
config.bn_weight_decay = False  
config.one_cycle = True         
config.learning_rate = 3e-3     # learning rate
save_model = True

wandb: Currently logged in as: riya-chougule (use `wandb login --relogin` to force relogin)


In [3]:
ignore_index=255

In [4]:
# Custom values to filter runs
config.training_stages = 2

# Data paths
path_data = Path('/content/drive/MyDrive/Kaggle/bdd100k_seg/bdd100k/seg')
path_lbl = path_data / 'labels'
path_img = path_data / 'images'

# Associate a label to an input
get_y_fn = lambda x: path_lbl / x.parts[-2] / f'{x.stem}_train_id.png'

In [5]:
segmentation_classes = [
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car',
    'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'void'
]
void_code = 19

In [6]:
src = (SegmentationItemList.from_folder(path_img).use_partial_data(0.01)
       .split_by_folder(train='train', valid='val')
       .label_from_func(get_y_fn, classes=segmentation_classes))

# Resize, augment, load in batch & normalize (so we can use pre-trained networks)
data = (src.transform(get_transforms(), size=config.img_size, tfm_y=True)
        .databunch(bs=config.batch_size)
        .normalize(imagenet_stats))

In [7]:
def acc(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask] == target[mask]).float().mean()

In [8]:
learn = unet_learner(
    data,
    arch=encoder,
    pretrained=config.pretrained,
    metrics=acc,
    wd=config.weight_decay,
    bn_wd=config.bn_weight_decay,
    callback_fns=partial(WandbCallback, save_model=save_model, monitor='acc', input_type='images'))

In [9]:
if config.one_cycle:
    learn.fit_one_cycle(config.epochs // 2,
    max_lr=slice(config.learning_rate))
    learn.unfreeze()
    learn.fit_one_cycle(config.epochs // 2,
        max_lr=slice(config.learning_rate / 100,config.learning_rate / 10))
else:
    learn.fit(config.epochs,lr=slice(config.learning_rate))

epoch,train_loss,valid_loss,acc,time


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


RuntimeError: ignored